- Note that the purpose of this course is to teach functional programming, with Scala providing a good platform to put both functional and Object oriented programming in practise

## Elements of Programming

- All proper languages provide
    1. Primitives - Representing simplest elements of the language (int, str)
    2. Ways to combine expressions (add 2 numbers, concat 2 strings)
    3. Ways to abstract expressions (name an expression and then we can refer to it by name)

- A non-primitive expression is evaluated by:
    - Take leftmost operator
    - Evaluate its operands (left before right)
    - Apply operator to operands

- In scala, this is the general way of defining a function. Not too far from Python

```scala
    def square(x: Double): Double = x * x

    def sumOfSquares(x: Double, y: Double): Double = square(x) + square(y)
```

- In Scala, primitives follow Java primitives
    | Type | Description |
    | --- | --- |
    | Int | 32 bit ints |
    | Long | 64 bit ints |
    | Float | 32 bit floats |
    | Double | 64 bit floats |
    | Char | 16-bit unicode chars |
    | Short | 16-bit integer |
    | Byte | 8-bit integer |
    | Boolean | true/false |

### Evaluation Basics

- General rules of evaluation
    - A name is evaluated by replacing the RHS of its definition
    - The evaluation process stops when it results in a value
    - A value is a number (let's just assume this for now)

- Evaluation of function applications (similar to evaluating operators)
    - Evaluate all function arguments from left to right
    - Replace the function application by the function's right hand side
    - Replace the formal parameters of the function by actual arguments

    - Example
        - sumOfSquares(3, 2+2)
        - sumOfSquares(3, 4)
        - sumOfSquares(3, 4)
        - square(3) + square(4)
        - 3*3 + square(4)
        - 9 + square(4)
        - 9 + 4*4
        - 9 + 16
        - 25

- Why is this important?
    - This is known as the **substitution model**, and the idea is that all evaluation simply reduces an expression to a value
    - As it turns out, we can express all algorithms using this model, so long as there are no side-effects (i.e. doesn't modify anything external to the function)
    - Formally, this is known as **$\lambda$-calculus**, which is the foundation for functional programming

- Termination
    - Does every expression reduce to a value in a finite number of steps?
    - No
    - `def loop: int = loop`

- Is this the only evaluation strategy possible? For example, rather than reducing 2+2 to 4 immediately, can we just pass 2+2 downwards?
    - Yes of course!
        - sumOfSquares(3, 2+2)
        - square(3) + square(2+2)
        - 3*3 + square(2+2)
        - 9 + square(2+2)
        - 9 + (2+2)*(2+2)
        - 9 + 4*4
        - 9 + 16
        - 25 
    - Passing in the unevaluated expression is known as **call-by-name**, while the former example is a **call-by-value**
    - Both strategies reach the same final value so long as the reduced expression consist of pure functions, and both evaluations terminate
    - Pros vs Cons
        - **call-by-name**: don't need to evaluate stuff that is not used in the function 
        - **call-by-value**: every function argument is evaluated only once

- Pop Quiz: Suppose you are given the function below. For each of the following function application, which strategy is faster; call-by-name (CBN) or call-by-value (CBV)?
    ```scala
        def test(x: Int, y: Int) = x * x
    ```

    - `test(2,3)`
        - Same. `Int` is a primitive, so no additional evaluation needed for either CBN/CBV
    - `test(3+4, 8)`
        - CBV; because you only need to evalute 3+4 once, vs twice is you CBN
    - `test(7, 2*4)`
        - CBN, because 2*4 is not evaluated in CBN
    - `test(3+4, 2*4)`
        - Same, because in CBV you save time evaluating 3+4 only once, but lose time by evaluated 2*4 which is not needed
    

### Evaluation Strategies and Termination

- We talked about "Call-By-Name" and "Call-By-Value" in the previous section, and how they reduce to the same value so long as termination is guaranteed

- But what if it is not?
    - Then CBN is guaranteed to terminate if CBV terminates
    - BUT the inverse is not true!!

- Let's see an example where termination occurs in CBN but not CBV
    ```scala
        def first(x: Int, y: Int): Int = x

        first(1, loop)
    ```
    - In CBN, `loop` is never evaluated, so it returns `x`. But in CBV, the program tries to evaluation `loop`, which never terminates

- In Scala, CBV is the default, BUT it allows you to specify whether to CBV or CBN!!
    ```scala
        def constOne_callYByName(x: Int, y => Int): Int = 1

        def constOne_callYByValue(x: Int, y: Int): Int = 1
    ```
    - In the first case, `=>` allows us to get Scala to call `y` by name!

- Scala also supports delayed evaluation of variables that are evaluated only when used. This is called `lazy val`

### Conditionals

- Conditional Expressions: `if-else`
    - This is similar to Java if-else, but works on expressions, not statements 
    - Actually the closest analogous expression from java would be `x >= 0 ? x: -x`
    - Jargon: `x >= 0` in the function below is the **predicate** of type Boolean
    ```scala
        def abs(x: Int) = if x >= 0 then x else -x
    ```

- Boolean Expressions
    - Can be composed of 
        ```scala
            true false  // Constants
            !b // Negation
            b && b // Conjunction
            b || b // Disjunction
        ```
    - Also has the usual comparison operators
        - > < >= <= == !=
    - Boolean expressions also have their own evaluation rules that come very close to Python
        ```scala
            !true // false
            !false // true
            true && e // e
            false && e // false
            true || e // true
            false || e // e
        ```
    - Note that && and || do not always require the RHS to be evaluated
        - That is, if you want `true || e`, it will return `true` without evaluating `e`
        - this is known as **short circuit evaluation**

### Value Definitions

- Just as function parameters can be passed by value or by name, the same distinction applies for definitions!
    - By name --> Use `def`
        - RHS is evaluated on use
    - By value --> use `val`
        - e.g. `val x = square(2)`
        - RHS is evaluated at the point of the definition

- To be clear what the difference is, let's examine the case where the RHS does not terminate
    ```scala
        def loop: Boolean = loop

        def x = loop // will work
        val x = loop // infinite loop
    ```

- POP QUIZ: Implement functions `and` and `or` such that 
    ```scala
        // and(x, y) == x && y
        // or(x, y) == x || y

        def and(x: Boolean, y => Boolean): Boolean = if x then y else false

        def or(x: Boolean, y => Boolean): Boolean = if x then true else y
    ```

- We pass $y$ by name instead of value, to allow expression to short circuit (i.e. lazily evaluate y)

### Exercise: Implement Square Root with Newton's Method

- We define a function `sqrt` by implementing successive approximations using Newton's Method

- Recap of Newton's Method: See your notes on Newton's Method under Mathematics

```scala
    def isGoodEnough(guess: Double, x: Double): Boolean =
      math.abs(x - (guess*guess)) < 0.001

    def isGoodEnoughPct(guess: Double, x: Double): Boolean =
      math.abs(1 - x/(guess*guess)) < 0.01
    
    def improve(guess: Double, x: Double): Double = 
      0.5 * (guess + (x/guess))

    def sqrtIter(guess: Double, x: Double): Double = //recursive functions need an explicit return type, optional for regular functions
      if isGoodEnough(guess, x) then guess
      else sqrtIter(improve(guess, x), x)

    def sqrt(x: Double) = sqrtIter(1.0, x)
```

### Blocks and Lexical Scope

- In the previous exercise